In [19]:
# tr198_codec.py
# ---------------------------------------------------------------------------
#  Build a single TR198-series ceiling-fan 26-bit payload
#
#  Frame layout (MSB→LSB, 26 bits)
#
#   25 … 13   12 11 10 9    8 7      6     5     4     3     2 1 0
# ┌──────────┬──────────┬──────┬────┬────┬────┬────┬──────────────┐
# │  Tx-ID    │  SPEED    │ DIR  │ T │ L │  U │  D │  (reserved)  │
# │ (13 bits) │ (4 bits)  │10/01│mrk│gt │dim↑│dim↓│   (0)         │
# └──────────┴──────────┴──────┴────┴────┴────┴────┴──────────────┘
#
#  T  (bit-6)  : timer marker (set on every timed-off frame)
#  L  (bit-5)  : LIGHT-toggle (1 = “toggle lamp”)
#  U  (bit-4)  : brightness-UP  (press-&-hold)
#  D  (bit-3)  : brightness-DOWN (press-&-hold)  [ U and D = 1 → dim-down ]
#
#  Timers set both bit-6 and a secondary bit:
#       2 h → bits 6+4  (0x50)   4 h → bits 6    (0x40)   8 h → bits 6+5 (0x60)
#
#  SPEED field (bits 12-9) — values seen so far
#      0   = OFF               5  = level-6
#      1-9 = fan levels 1-9    8  = level-9     9 = level-10 (some receivers)
#     13   = breeze-1          14 = breeze-3    (we expect 15 = breeze-2)
# ---------------------------------------------------------------------------

from typing import Literal, Optional

Dir = Literal["forward", "reverse"]
Timer = Optional[Literal[2, 4, 8]]          # hours
Dim   = Optional[Literal["up", "down"]]
Breeze = Optional[Literal[1, 2, 3]]         # natural-wind modes

def _direction_bits(direction: Dir) -> int:      # bits 8-7
    return 0b10 if direction.startswith("f") else 0b01          # 10=fwd, 01=rev

def _speed_bits(speed: int | None, breeze: Breeze) -> int:      # bits 12-9
    if breeze:
        breeze_map = {1: 0b1101, 2: 0b1111, 3: 0b1110}
        return breeze_map[breeze]
    if speed is None:
        raise ValueError("speed must be 0-9 when breeze is None")
    if not 0 <= speed <= 9:
        raise ValueError("speed must be 0-9")
    return speed

def _low_bits(light_toggle: bool, dim: Dim, timer: Timer) -> int:
    low = 0
    if light_toggle:                 # bit-5
        low |= 0x20
    if dim:
        if dim == "up":              # bit-4
            low |= 0x10
        elif dim == "down":          # bits 4+3
            low |= 0x18
    if timer:
        if timer == 2:               # bits 6+4
            low |= 0x50
        elif timer == 4:             # bit-6 only
            low |= 0x40
        elif timer == 8:             # bits 6+5
            low |= 0x60
    return low & 0x7F                # keep bottom 7 bits

def build_payload(
    tx_id: int,
    *,
    speed: int | None = None,
    direction: Dir = "reverse",
    light_toggle: bool = False,
    dim: Dim = None,
    timer: Timer = None,
    breeze: Breeze = None
) -> int:
    """
    Return a **26-bit** TR198 payload as an int.

    tx_id      : 13-bit handset address (0-0x1FFF)
    speed      : 0-9  (ignored if `breeze` is supplied)
    direction  : 'forward' | 'reverse'
    light_toggle : True to toggle lamp
    dim        : 'up' | 'down'  (hold to ramp)
    timer      : 2 | 4 | 8  (hours)
    breeze     : 1 | 2 | 3  (natural wind mode)
    """
    if not 0 <= tx_id <= 0x1FFF:
        raise ValueError("tx_id must fit in 13 bits (0-0x1FFF)")

    payload = 0
    payload |= tx_id             << 10
    payload |= _speed_bits(speed, breeze) << 6
    payload |= _direction_bits(direction) << 4
    payload |= _low_bits(light_toggle, dim, timer)

    return payload


# ─── helpers you’ll usually want ───────────────────────────────────────────

def payload_bin(payload: int) -> str:
    "26-bit binary string (no sync mark)."
    return format(payload, "023b")

def payload_hex(payload: int) -> str:
    "0x… hex literal."
    return hex(payload)

def bitstream(payload: int, repeats: int = 1) -> str:
    """
    Full OOK bitstream with the **sync '1'** prefixed, repeated `repeats` times.
    This is what you’d feed to rpitx, CC1101 burst mode, etc.
    """
    frame = payload_bin(payload)          # sync + 26 data bits
    return frame * repeats


# ─── sanity-checks against the samples the user captured ───────────────────
MY_ID = 0x15A9                  # 1010110101001  (your handset’s address)

# Direction-up (forward) @ speed-9
print("forward 9  →", payload_bin(
    build_payload(MY_ID, speed=8, direction="forward")))

# Direction-down (reverse) @ speed-9  (matches 'direction down' & 'speed nine')
print("reverse 9  →", payload_bin(
    build_payload(MY_ID, speed=8, direction="reverse")))

# Power OFF  (speed = 0)
print("fan off    →", payload_bin(
    build_payload(MY_ID, speed=0)))

# Light toggle
print("light      →", payload_bin(
    build_payload(MY_ID, speed=0, light_toggle=True)))

# Timer 2 h (IDLE speed so fan stays at level-1, reverse dir)
print("timer 2 h  →", payload_bin(
    build_payload(MY_ID, speed=1, timer=2)))

# Brightness DOWN (hold)
print("dim down   →", payload_bin(
    build_payload(MY_ID, speed=0, dim="down")))

# Breeze mode-3 with fan otherwise off
print("breeze-3   →", payload_bin(
    build_payload(MY_ID, speed=0, breeze=3)))

forward 9  → 10101101010011000100000
reverse 9  → 10101101010011000010000
fan off    → 10101101010010000010000
light      → 10101101010010000110000
timer 2 h  → 10101101010010001010000
dim down   → 10101101010010000011000
breeze-3   → 10101101010011110010000


In [34]:
# tr198_codec_23bit.py  –  sync (1) + 23-bit payload = 24 symbols on the air
from typing import Literal, Optional

Dir     = Literal["forward", "reverse"]
DimDir  = Optional[Literal["up", "down"]]
Breeze  = Optional[Literal[1, 2, 3]]        # natural-wind modes
Timer   = Optional[Literal[2, 4, 8]]        # hours

def _dir_bits(direction: Dir) -> int:               # bits 5-4
    return 0b10 if direction.startswith("f") else 0b01

def _speed_bits(speed: int | None, breeze: Breeze) -> int:   # bits 9-6
    if breeze:
        return {1: 0b1011, 2: 0b1111, 3: 0b1101}[breeze]
    if speed is None:
        raise ValueError("speed (0-10) required when breeze is None")
    if not 0 <= speed <= 10:
        raise ValueError("speed out of range 0-10")
    return speed

def _low_bits(light: bool, dim: DimDir, timer: Timer) -> int:
    """bits 3-0 : see table above"""
    b3 = 1 if timer else 0
    b2 = 1 if (timer == 8 or (not timer and light)) else 0
    b1 = 1 if (timer == 2 or (not timer and dim)) else 0
    b0 = 1 if (dim == "down") else 0
    return (b3 << 3) | (b2 << 2) | (b1 << 1) | b0       # 0-15

def build_payload(
        tx_id: int,
        *,
        speed: int | None = None,
        direction: Dir = "reverse",
        light_toggle: bool = False,
        dim: DimDir = None,
        timer: Timer = None,
        breeze: Breeze = None,
) -> int:
    """
    Return the **23-bit** payload (no sync, no trailing zeros).
    """
    if not 0 <= tx_id <= 0x1FFF:
        raise ValueError("tx_id must fit in 13 bits (0-0x1FFF)")

    payload  = (tx_id                         << 10)
    payload |= (_speed_bits(speed, breeze)    <<  6)
    payload |= (_dir_bits(direction)          <<  4)
    payload |= _low_bits(light_toggle, dim, timer)

    return payload & 0x7FFFFF                 # mask to 23 bits

def build_pairing_payload(tx_id: int) -> int:
    """
    Return the **23-bit** pairing payload (no sync, no trailing zeros).
    This is used to pair a new handset with a receiver.
    """
    if not 0 <= tx_id <= 0x1FFF:
        raise ValueError("tx_id must fit in 13 bits (0-0x1FFF)")
    pairing_bits = 0b1111000000  # 10 bits of '1's, rest are zeroes
    # Pairing payload is just the Tx-ID in bits 22-10, rest are zeroes
    return (tx_id << 10) | pairing_bits

# ── helpers ────────────────────────────────────────────────────────────────
bin23 = lambda n: format(n, "023b")
hex23 = lambda n: hex(n)

def bitstream(payload: int, repeats: int = 6) -> str:
    """sync '1' + 23 data bits, repeated N times (no gap zeros)."""
    frame = "1" + bin23(payload)
    return frame * repeats

# ── quick self-test against your examples ─────────────────────────────────
ID = 0x15A9                                # 1010110101001

tests = {
    "fan off"      : build_payload(ID, speed=0),
    "dir down"     : build_payload(ID, speed=1, direction="reverse"),
    "dir up"       : build_payload(ID, speed=1, direction="forward"),
    "light toggle" : build_payload(ID, speed=0, light_toggle=True),
    "speed-1"      : build_payload(ID, speed=1),
    "speed-2"      : build_payload(ID, speed=2),
    "speed-3"      : build_payload(ID, speed=3),
    "speed-4"      : build_payload(ID, speed=4),
    "speed-5"      : build_payload(ID, speed=5),
    "speed-6"      : build_payload(ID, speed=6),
    "speed-7"      : build_payload(ID, speed=7),
    "speed-8"      : build_payload(ID, speed=8),
    "speed-9"      : build_payload(ID, speed=9),
    "breeze-1"    : build_payload(ID, breeze=1),
    "breeze-2"     : build_payload(ID, breeze=2),
    "breeze-3"     : build_payload(ID, breeze=3),
    "timer 2 h"    : build_payload(ID, speed=1, timer=2),
    "timer 8 h"    : build_payload(ID, speed=1, timer=8),
    "dim up"       : build_payload(ID, speed=0, dim="up"),
    "dim down"     : build_payload(ID, speed=0, dim="down"),
    "pairing"      : build_pairing_payload(ID),
}
for label, p in tests.items():
    print(f"{label:12s} → {bin23(p)}")

fan off      → 10101101010010000010000
dir down     → 10101101010010001010000
dir up       → 10101101010010001100000
light toggle → 10101101010010000010100
speed-1      → 10101101010010001010000
speed-2      → 10101101010010010010000
speed-3      → 10101101010010011010000
speed-4      → 10101101010010100010000
speed-5      → 10101101010010101010000
speed-6      → 10101101010010110010000
speed-7      → 10101101010010111010000
speed-8      → 10101101010011000010000
speed-9      → 10101101010011001010000
breeze-1     → 10101101010011011010000
breeze-2     → 10101101010011111010000
breeze-3     → 10101101010011101010000
timer 2 h    → 10101101010010001011010
timer 8 h    → 10101101010010001011100
dim up       → 10101101010010000010010
dim down     → 10101101010010000010011
pairing      → 10101101010011111000000


In [ ]:
10101101010010000010100
10101101010010000010100
10101101010010000110000

10101101010010000110000